## Step 1: Setup

In [ ]:
## Lots of imports

# Install Ultralytics YOLOv8
!pip install ultralytics
!pip install torchvision

# Import necessary libraries
import torch
import os
from ultralytics import YOLO
from torch.utils.data import DataLoader, Dataset
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.2/800.2 kB 5.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

Drive Setup

In [ ]:
## If you did this correctly you should see here "drive" and "sample_data"
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
!ls

Mounted at /content/drive
drive  sample_data


In [ ]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


## Step 2: Data Import and Preperation

In [ ]:
root = "/content/drive/MyDrive/AISS - CV/2. Code/Model/Data_Reviewed"
images_folder = os.path.join(root, 'images')
labels_folder = os.path.join(root, 'labels')

Build Transformers

In [ ]:
train_val_transform = transforms.Compose([transforms.Resize(224),
                           transforms.RandomRotation(6),
                           transforms.RandomHorizontalFlip(0.4),
                           transforms.RandomCrop(224, padding = 8),
                           transforms.ToTensor()
                       ])

test_transform = transforms.Compose([
                           transforms.Resize(224),
                           transforms.CenterCrop(224),
                           transforms.ToTensor()
                       ])

Create datasets from Google Drive using Transformers

In [ ]:
counter = 0

# List all .txt files in the specified folder
label_files = [f for f in os.listdir(labels_folder) if f.endswith('.txt')]

# Count the number of label files
for label in label_files:
    counter += 1

# Print the count
print(counter)

1225


In [ ]:
# ChatGPT Version mit Cleanup

import os
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

# Define transformations
train_val_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Custom function to load images and labels
def load_dataset(images_folder, labels_folder, transform=None):
    image_files = [f for f in os.listdir(images_folder) if f.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp'))]
    dataset = []

    for image_file in image_files:
        # Load image
        image_path = os.path.join(images_folder, image_file)
        image = Image.open(image_path).convert("RGB")

        if transform:
            image = transform(image)

        # Load corresponding label
        label_file = os.path.splitext(image_file)[0] + ".txt"
        label_path = os.path.join(labels_folder, label_file)

        if not os.path.exists(label_path):
            continue

        boxes = []
        labels = []
        with open(label_path, 'r') as f:
            for line in f.readlines():
                class_id, center_x, center_y, width, height = map(float, line.strip().split())
                boxes.append([center_x, center_y, width, height])
                labels.append(int(class_id))

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        dataset.append((image, {'boxes': boxes, 'labels': labels}))

    return dataset

# Function to clean up labels without corresponding images
def clean_labels(images_folder, labels_folder):
    image_files = {os.path.splitext(f)[0] for f in os.listdir(images_folder) if f.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp'))}
    label_files = {os.path.splitext(f)[0] for f in os.listdir(labels_folder) if f.endswith('.txt')}

    # Find label files without corresponding image files and remove them
    labels_to_remove = label_files - image_files
    for label in labels_to_remove:
        label_path = os.path.join(labels_folder, label + ".txt")
        os.remove(label_path)

    return len(label_files) - len(labels_to_remove)

# Set the directories for images and labels
root = '/content/drive/MyDrive/AISS - CV/2. Code/Model/Data_Reviewed'
images_folder = os.path.join(root, 'images')
labels_folder = os.path.join(root, 'labels')

# Clean up the labels
remaining_labels = clean_labels(images_folder, labels_folder)
print(f"The number of remaining labels is {remaining_labels}")

# Load the train/validation dataset
train_val_dataset = load_dataset(images_folder, labels_folder, transform=train_val_transform)

# Print the size of the train/validation dataset
print(f"The train/validation dataset contains {len(train_val_dataset)} labeled images")

# Example usage with DataLoader
train_val_loader = DataLoader(train_val_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

# Iterate through the DataLoader
for images, targets in train_val_loader:
    print(f"Batch of images: {len(images)}")
    print(f"Batch of targets: {len(targets)}")
    break


The number of remaining labels is 1225
The train/validation dataset contains 1225 labeled images
Batch of images: 4
Batch of targets: 4


In [ ]:
# Old Version - do not run (failed due to duplicates)

import os
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

# Define transformations
train_val_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Custom function to load images and labels
def load_dataset(images_folder, labels_folder, transform=None):
    image_files = [f for f in os.listdir(images_folder) if f.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp'))]
    dataset = []

    for image_file in image_files:
        # Load image
        image_path = os.path.join(images_folder, image_file)
        image = Image.open(image_path).convert("RGB")

        if transform:
            image = transform(image)

        # Load corresponding label
        label_file = os.path.splitext(image_file)[0] + ".txt"
        label_path = os.path.join(labels_folder, label_file)

        boxes = []
        labels = []
        with open(label_path, 'r') as f:
            for line in f.readlines():
                class_id, center_x, center_y, width, height = map(float, line.strip().split())
                boxes.append([center_x, center_y, width, height])
                labels.append(int(class_id))

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        dataset.append((image, {'boxes': boxes, 'labels': labels}))

    return dataset

# Set the directories for images and labels
root = '/content/drive/MyDrive/AISS - CV/2. Code/Model/Data_Reviewed'
images_folder = os.path.join(root, 'images')
labels_folder = os.path.join(root, 'labels')

# Load the train/validation dataset
train_val_dataset = load_dataset(images_folder, labels_folder, transform=train_val_transform)

# Print the size of the train/validation dataset
print(f"The train/validation dataset contains {len(train_val_dataset)} labeled images")

# Example usage with DataLoader
train_val_loader = DataLoader(train_val_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

# Iterate through the DataLoader
for images, targets in train_val_loader:
    print(f"Batch of images: {len(images)}")
    print(f"Batch of targets: {len(targets)}")
    break


Create Dataloaders

In [ ]:
#train_loader = DataLoader(train_val_dataset, batch_size=16, shuffle=True, num_workers=4)
#val_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

## Step 3: Pre-trained Model Load

In [ ]:
%cd /content/drive/MyDrive/AISS - CV/2. Code/Model
!yolo task=detect mode=train model=yolov8s.pt data= data.yaml epochs=25 imgsz=224 plots=True

/content/drive/MyDrive/AISS - CV/2. Code/Model
Ultralytics YOLOv8.2.48 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_lab

Inferece

In [ ]:
!yolo task=detect mode=predict model='/content/drive/MyDrive/AISS - CV/2. Code/Model/runs/detect/train9/weights/best.pt' source='/content/drive/MyDrive/AISS - CV/3. Daten/Vasilis' imgsz=224 save=True show=True save_txt=True

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Ultralytics YOLOv8.2.55 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,127,906 parameters, 0 gradients, 28.4 GFLOPs

image 1/66 /content/drive/MyDrive/AISS - CV/3. Daten/Vasilis/IMG_0812.jpeg: 224x192 1 helmet, 1 person_with_full_safety, 1 jacket, 117.0ms
image 2/66 /content/drive/MyDrive/AISS - CV/3. Daten/Vasilis/IMG_0813.jpeg: 224x192 1 helmet, 1 person_with_full_safety, 1 jacket, 10.8ms
image 3/66 /content/drive/MyDrive/AISS - CV/3. Daten/Vasilis/IMG_0814.jpeg: 224x192 1 helmet, 1 person_with_full_safety, 1 person_only_with_jacket, 1 jacket, 12.0ms
image 4/66 /content/drive/MyDrive/AISS - CV/3. Daten/Vasilis/IMG_0815.jpeg: 224x192 1 helmet, 1 person_with_full_safety, 1 jacket, 9.6ms
image 5/66 /content/drive/MyDrive/AISS - CV/3. Daten/Vasilis/IMG_0816.jpeg: 224x192 1 helmet, 1 jacket, 7.7ms
image 6/66 /content/drive/MyDrive/AISS - CV/3. Daten/Vasilis/IMG_0

In [ ]:
# Inference Vasilis
!yolo task=detect mode=predict model='/content/drive/MyDrive/AISS - CV/2. Code/Model/runs/detect/train9/weights/best.pt' source='/content/drive/MyDrive/AISS - CV/3. Daten/Vasilis' imgsz=224 save=True show=True save_txt=True project='/content/drive/MyDrive/AISS - CV/Vasilis_results' name='detection_output'

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Ultralytics YOLOv8.2.55 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,127,906 parameters, 0 gradients, 28.4 GFLOPs

image 1/66 /content/drive/MyDrive/AISS - CV/3. Daten/Vasilis/IMG_0812.jpeg: 224x192 1 helmet, 1 person_with_full_safety, 1 jacket, 72.5ms
image 2/66 /content/drive/MyDrive/AISS - CV/3. Daten/Vasilis/IMG_0813.jpeg: 224x192 1 helmet, 1 person_with_full_safety, 1 jacket, 8.5ms
image 3/66 /content/drive/MyDrive/AISS - CV/3. Daten/Vasilis/IMG_0814.jpeg: 224x192 1 helmet, 1 person_with_full_safety, 1 person_only_with_jacket, 1 jacket, 7.5ms
image 4/66 /content/drive/MyDrive/AISS - CV/3. Daten/Vasilis/IMG_0815.jpeg: 224x192 1 helmet, 1 person_with_full_safety, 1 jacket, 7.4ms
image 5/66 /content/drive/MyDrive/AISS - CV/3. Daten/Vasilis/IMG_0816.jpeg: 224x192 1 helmet, 1 jacket, 7.5ms
image 6/66 /content/drive/MyDrive/AISS - CV/3. Daten/Vasilis/IMG_0817

In [ ]:
# Inference Isi
!yolo task=detect mode=predict model='/content/drive/MyDrive/AISS - CV/2. Code/Model/runs/detect/train9/weights/best.pt' source='/content/drive/MyDrive/AISS - CV/3. Daten/Isabelle/AISS-CV_Isabelle' imgsz=224 save=True show=True save_txt=True project='/content/drive/MyDrive/AISS - CV/Isi_results' name='detection_output'

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Ultralytics YOLOv8.2.55 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,127,906 parameters, 0 gradients, 28.4 GFLOPs

image 1/104 /content/drive/MyDrive/AISS - CV/3. Daten/Isabelle/AISS-CV_Isabelle/IMG_1248.jpeg: 224x192 1 helmet, 1 person_only_with_helmet, 128.0ms
image 2/104 /content/drive/MyDrive/AISS - CV/3. Daten/Isabelle/AISS-CV_Isabelle/IMG_1249.jpeg: 224x192 1 helmet, 1 person_only_with_helmet, 8.5ms
image 3/104 /content/drive/MyDrive/AISS - CV/3. Daten/Isabelle/AISS-CV_Isabelle/IMG_1250.jpeg: 224x192 1 helmet, 1 person_only_with_helmet, 8.3ms
image 4/104 /content/drive/MyDrive/AISS - CV/3. Daten/Isabelle/AISS-CV_Isabelle/IMG_1251.jpeg: 224x192 1 helmet, 1 person_only_with_helmet, 7.9ms
image 5/104 /content/drive/MyDrive/AISS - CV/3. Daten/Isabelle/AISS-CV_Isabelle/IMG_1252.jpeg: 224x192 1 helmet, 1 person_only_with_helmet, 8.4ms
image 6/104 /content/dr

In [ ]:
# Inference Luis
!yolo task=detect mode=predict model='/content/drive/MyDrive/AISS - CV/2. Code/Model/runs/detect/train9/weights/best.pt' source='/content/drive/MyDrive/AISS - CV/3. Daten/Luis/AISS-CV_Luis' imgsz=224 save=True show=True save_txt=True project='/content/drive/MyDrive/AISS - CV/Luis_results' name='detection_output'

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Ultralytics YOLOv8.2.55 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,127,906 parameters, 0 gradients, 28.4 GFLOPs

image 1/174 /content/drive/MyDrive/AISS - CV/3. Daten/Luis/AISS-CV_Luis/IMG_1074.jpeg: 224x192 1 helmet, 1 person_with_full_safety, 1 jacket, 249.6ms
image 2/174 /content/drive/MyDrive/AISS - CV/3. Daten/Luis/AISS-CV_Luis/IMG_1075.jpeg: 224x192 1 helmet, 1 person_with_full_safety, 1 jacket, 14.8ms
image 3/174 /content/drive/MyDrive/AISS - CV/3. Daten/Luis/AISS-CV_Luis/IMG_1076.jpeg: 224x192 1 helmet, 1 person_with_full_safety, 1 jacket, 15.1ms
image 4/174 /content/drive/MyDrive/AISS - CV/3. Daten/Luis/AISS-CV_Luis/IMG_1077.jpeg: 224x192 1 helmet, 1 person_with_full_safety, 1 jacket, 7.7ms
image 5/174 /content/drive/MyDrive/AISS - CV/3. Daten/Luis/AISS-CV_Luis/IMG_1078.jpeg: 224x192 1 helmet, 1 person_with_full_safety, 1 jacket, 7.7ms
image 6/174

In [ ]:
# Inference Daniel
!yolo task=detect mode=predict model='/content/drive/MyDrive/AISS - CV/2. Code/Model/runs/detect/train9/weights/best.pt' source='/content/drive/MyDrive/AISS - CV/3. Daten/Daniel' imgsz=224 save=True show=True save_txt=True project='/content/drive/MyDrive/AISS - CV/Daniel_results' name='detection_output'

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Ultralytics YOLOv8.2.55 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,127,906 parameters, 0 gradients, 28.4 GFLOPs

image 1/104 /content/drive/MyDrive/AISS - CV/3. Daten/Daniel/IMG_20240531_142354.jpg: 192x224 1 helmet, 1 person_with_full_safety, 1 jacket, 75.8ms
image 2/104 /content/drive/MyDrive/AISS - CV/3. Daten/Daniel/IMG_20240531_142355.jpg: 192x224 1 helmet, 1 person_with_full_safety, 2 person_only_with_jackets, 1 jacket, 9.1ms
image 3/104 /content/drive/MyDrive/AISS - CV/3. Daten/Daniel/IMG_20240531_142355__01.jpg: 192x224 1 helmet, 1 person_with_full_safety, 1 person_only_with_jacket, 3 jackets, 7.7ms
image 4/104 /content/drive/MyDrive/AISS - CV/3. Daten/Daniel/IMG_20240531_142357.jpg: 192x224 1 helmet, 1 person_with_full_safety, 2 person_only_with_jackets, 1 jacket, 7.9ms
image 5/104 /content/drive/MyDrive/AISS - CV/3. Daten/Daniel/IMG_20240531_1424

In [ ]:
import os
import cv2
from ultralytics import YOLO

# Load the fine-tuned YOLOv8 model
model = YOLO('/content/drive/MyDrive/AISS - CV/2. Code/Model/runs/detect/train9/weights/best.pt')  # Adjust path as needed

# Path to the folder containing new images
image_folder = '/content/drive/MyDrive/AISS - CV/3. Daten/Vasilis'  # Adjust path as needed

# Path to save the inferred images and annotations
output_folder = '/content/drive/MyDrive/AISS - CV/3. Daten/Inferred_Images_Vasilis_2'
annotations_folder = '/content/drive/MyDrive/AISS - CV/3. Daten/Inferred_Annotations_Vasilis_2'

os.makedirs(output_folder, exist_ok=True)
os.makedirs(annotations_folder, exist_ok=True)

# Function to create a YOLO-compatible annotation
def create_yolo_annotation(image_path, results, output_annotation_path):
    img = cv2.imread(image_path)
    h, w, _ = img.shape

    annotation_lines = []
    for result in results:
        for box in result.boxes:
            # Ensure there are enough values to unpack
            if len(box) >= 6:
                x_min, y_min, x_max, y_max, conf, cls = box[:6]

                # Calculate normalized coordinates
                x_center = (x_min + x_max) / 2.0 / w
                y_center = (y_min + y_max) / 2.0 / h
                width = (x_max - x_min) / w
                height = (y_max - y_min) / h

                # Create a line for the annotation file
                annotation_line = f"{int(cls)} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
                annotation_lines.append(annotation_line)

    # Save the annotation to file
    with open(output_annotation_path, 'w') as f:
        f.write("\n".join(annotation_lines))

    # Print the annotation to console
    print(f"Annotations for {image_path}:\n" + "\n".join(annotation_lines))

# Perform inference on new images and save results
def perform_inference(image_path, output_folder, annotations_folder):
    img = cv2.imread(image_path)
    results = model(img)

    # Debugging: Print the results
    print(f"Performing inference on {image_path}")

    # Save the image with detections
    output_image_path = os.path.join(output_folder, os.path.basename(image_path))
    annotated_img = results[0].plot(show=False)  # Updated to not display the image
    cv2.imwrite(output_image_path, annotated_img)

    # Create YOLO-compatible annotation
    output_annotation_path = os.path.join(annotations_folder, os.path.splitext(os.path.basename(image_path))[0] + '.txt')
    create_yolo_annotation(image_path, results, output_annotation_path)

# Iterate over images in the folder and perform inference
for img_file in os.listdir(image_folder):
    if img_file.endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(image_folder, img_file)
        perform_inference(img_path, output_folder, annotations_folder)

print(f"Inference completed. Images and annotations saved to {output_folder} and {annotations_folder}")




0: 224x192 1 helmet, 1 person_with_full_safety, 1 jacket, 9.7ms
Speed: 1.1ms preprocess, 9.7ms inference, 1.8ms postprocess per image at shape (1, 3, 224, 192)
Performing inference on /content/drive/MyDrive/AISS - CV/3. Daten/Vasilis/IMG_0812.jpeg
Annotations for /content/drive/MyDrive/AISS - CV/3. Daten/Vasilis/IMG_0812.jpeg:
<annotation><folder>Vasilis</folder><filename>IMG_0812.jpeg</filename><path>/content/drive/MyDrive/AISS - CV/3. Daten/Vasilis/IMG_0812.jpeg</path><source><database>Unknown</database></source><size><width>3024</width><height>4032</height><depth>3</depth></size><segmented>0</segmented></annotation>

0: 224x192 1 helmet, 1 person_with_full_safety, 1 jacket, 8.8ms
Speed: 1.5ms preprocess, 8.8ms inference, 2.1ms postprocess per image at shape (1, 3, 224, 192)
Performing inference on /content/drive/MyDrive/AISS - CV/3. Daten/Vasilis/IMG_0813.jpeg
Annotations for /content/drive/MyDrive/AISS - CV/3. Daten/Vasilis/IMG_0813.jpeg:
<annotation><folder>Vasilis</folder><filen

## Step 4: Definition of Loss Function and Optimizer

In [ ]:
# Define optimizer and learning rate scheduler
# YOLOv8 typically manages these configurations, but we can override them
optimizer = torch.optim.SGD(model.model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

## Step 5: Model Training

## Step 6: Model Evaluation

In [ ]:
# Evaluate the model
model.eval()
corrects = 0
total = 0

with torch.no_grad():
    for images, targets in val_loader:
        images = images.to(device)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss, outputs = model.validation_step((images, targets))
        corrects += sum([torch.sum(out['labels'] == targ['labels']).item() for out, targ in zip(outputs, targets)])
        total += len(targets)

print('Validation Accuracy: {:.4f}'.format(corrects / total))

# Visualize some predictions
model.eval()
images, targets = next(iter(val_loader))
images = images.to(device)
outputs = model(images)
for img, target, output in zip(images, targets, outputs):
    img = img.cpu().numpy().transpose(1, 2, 0)
    plt.imshow(img)
    plt.show()
    print("Target:", target['labels'])
    print("Output:", output['labels'])

NameError: name 'model' is not defined